In [ ]:
import sys
sys.path.append("../")

In [ ]:
from torch.optim import Adam
import cv2

In [ ]:
import torch
import torch.nn.functional as F
from colour.plotting import *
import numpy as np
import matplotlib.pyplot as plt
from datasets.cave_dataset import CAVEDataset


In [ ]:
from datasets.cave_dataset import R

In [ ]:
dataset = CAVEDataset("../datasets/data/CAVE", None, mode="train")

In [ ]:
model = torch.nn.Linear(3, 31, bias=False)
# model.weight = torch.nn.Parameter(R.T)
optimizer = Adam(model.parameters(), lr=1e-3)
model.train()
mse = torch.nn.MSELoss()


In [ ]:
c, x_old, y, z, x_gt, lz, idx = dataset[0]

In [ ]:
R.shape

In [ ]:
n_epochs = 50

In [ ]:
model_path = "../artifacts/R.pt"

In [ ]:
baseline_total = 0
# lets check mse loss for R provided by other vendors
for items in dataset:
    c, x_old, y, z, x_gt, lz, idx = items
    z_ipt = z.numpy()
    Zd = torch.zeros(z.shape[0], y.shape[1], y.shape[2])
    C, N1, N2 = Zd.shape
    for c in range(Zd.shape[0]):
        Zd[c, :, :] = torch.FloatTensor(cv2.resize(z_ipt[c, :, :], (N1, N2), interpolation=cv2.INTER_CUBIC))
    
    y_pred = (R.T @ Zd.reshape(C, -1)).reshape(y.shape[0], N1, N2)
    loss = mse(y_pred, y)
    baseline_total += loss.item()

print(baseline_total/len(dataset))

In [ ]:
print(baseline_total/len(dataset))

In [ ]:
model.train()
best_mse_loss = baseline_total/len(dataset)
for epoch in range(n_epochs):
    total_mse_loss = 0
    for items in dataset:
        optimizer.zero_grad()
        c, x_old, y, z, x_gt, lz, idx = items
        z_ipt = z.numpy()
        Zd = torch.zeros(z.shape[0], y.shape[1], y.shape[2])
        C, N1, N2 = Zd.shape
        for c in range(Zd.shape[0]):
            Zd[c, :, :] = torch.FloatTensor(cv2.resize(z_ipt[c, :, :], (N1, N2), interpolation=cv2.INTER_CUBIC))
        Zd = Zd.permute(1, 2, 0).reshape(-1, C)[None, ...]
        y_pred = model(Zd)
        # y_pred = (newR @ Zd.reshape(C, -1)).reshape(*y.shape)
        y_pred = y_pred.transpose(1, 2).reshape(1, y.shape[0], N1, N2)
        loss = mse(y_pred, y[None, ...])
        loss.backward()
        optimizer.step()
        total_mse_loss += loss.item()

    print(f"epoch {epoch} mse: {total_mse_loss/len(dataset)}")
    # if total_mse_loss/len(dataset) < best_mse_loss:
    #     best_mse_loss =  total_mse_loss/len(dataset)
    #     print("saving ...")
    #     torch.save(model.state_dict(), model_path)

In [ ]:
torch.save(model.state_dict(), model_path)

In [ ]:
# z should be converted to have 31 bands
y[:,0,0]

In [ ]:
from models.metrics import (
    compare_mpsnr,
    compare_mssim,
    find_rmse,
    # compare_sam,
    compare_ergas
)
from torchmetrics import SpectralAngleMapper
from torchmetrics import ErrorRelativeGlobalDimensionlessSynthesis as ERGAS
sam = SpectralAngleMapper()
ergas = ERGAS(ratio=1/8)


In [ ]:
model.eval()

In [ ]:
R = model.weight

In [ ]:
R.shape

In [ ]:
R @z.reshape(z)

In [ ]:
test_dataset = CAVEDataset("../datasets/data/CAVE", None, mode="test")
test_loader = torch.utils.data.DataLoader(test_dataset,
                                        shuffle=False, batch_size=1)

In [ ]:
sf = 8

In [ ]:
# lets check all the metrics from the learned R


total_psnr, total_ssim, total_rmse, total_sam, total_ergas =0,0,0,0,0
with torch.no_grad():
    for items in test_dataset:
        c, x_old, y, z, x, lz, idx = items

        zc, N1, N2 = z.shape 
        x2 = (R @ z.reshape(zc, -1)).reshape(y.shape[0], N1, N2)
    
        x = x.squeeze()
        x2 = x2.squeeze()
        x = x.permute(1, 2, 0).detach().cpu().numpy()
        x2 = x2.permute(1, 2, 0).detach().cpu().numpy()
        
        total_ssim += compare_mssim(x, x2)
        rmse,  mse, rmse_per_band = find_rmse(x, x2)
        total_rmse += rmse
        total_psnr += compare_mpsnr(x, x2, mse)
        s
        total_ergas += compare_ergas(x, x2, sf, rmse_per_band)
        # total_sam += compare_sam(x, x2)
        # total_ergas += ergas(torch.from_numpy(x).permute(2, 0, 1)[None, ...], 
        #                     torch.from_numpy(x2).permute(2, 0, 1)[None, ...])
        # total_ergas += compare_ergas(x, x2,1/sf, rmse_per_band)

opt = f"""## Metric scores:
psnr:{total_psnr/len(test_loader)},
ssim:{total_ssim/len(test_loader)},
rmse:{total_rmse/len(test_loader)},
sam:{total_sam/len(test_loader)},
ergas:{total_ergas/len(test_loader)},
"""
print(opt)

In [ ]:
z.shape

In [ ]:
z_ = R.T @ (z.reshape(3, -1)/255)

In [ ]:
z_.min(), z_.max()

In [ ]:
zm = z.reshape(3, -1)
z_ = R.T @ ((zm - zm.min(-1).values[:, None] / zm.max(-1).values[:, None]))
z_.min(), z_.max()

In [ ]:
z_ = R.T @ (z.reshape(3, -1)/z.reshape(3, -1).max(1).values[:, None])
z_.min(), z_.max()

In [ ]:
y_ = R @ (y.reshape(31, -1)/y.reshape(31, -1).max(1).values[:, None])

In [ ]:

y_.min(), y_.max()

In [ ]:
y_.min(), y_.max()

In [ ]:
z_.min(), z_.max()

In [ ]:
# downsample 512, 512, 3 to 64, 64, 3

# def spectral_response_to_band(y, z):
    

In [ ]:
# import pyspectral
# pyspectral.rsr_reader.RelativeSpectralResponse() # wavelength to band number calculation

In [ ]:
# we need to have dowsampling loss better than L2(downsample(z, bicubic))

